In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob
import cv2
import os

In [16]:
def align_images(ref, fore):
    # Convert images to grayscale
    ref_gray = cv2.cvtColor((ref * 255).clip(0,255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    fore_gray = cv2.cvtColor((fore * 255).clip(0,255).astype(np.uint8), cv2.COLOR_BGR2GRAY)

    # Use ORB (Oriented FAST and Rotated BRIEF) to detect and compute keypoints and descriptors
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(ref_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(fore_gray, None)

    # Use the BFMatcher to find the best matches between the descriptors
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort the matches based on their distances
    matches = sorted(matches, key=lambda x: x.distance)

    # Get the corresponding points in both images
    ref_points = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    fore_points = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Use the RANSAC algorithm to estimate an affine transformation
    M, _ = cv2.estimateAffinePartial2D(fore_points, ref_points)

    # Apply the transformation to the fore image
    aligned_fore = cv2.warpAffine(fore, M, (ref.shape[1], ref.shape[0]))
    
    refCopy = ref.copy()
    mask = aligned_fore.sum(2) == 0
    mask = cv2.dilate(mask.astype(np.uint8), np.ones((3, 3), np.uint8), iterations=1).astype(bool)
    refCopy[~mask] = aligned_fore[~mask]

    return refCopy

def get_result(img_id, method):
    if method == "baseline":
        img = plt.imread(f"baseline/results/{img_id}.png")[:,:,:3]
    elif method == "ShadowGP":
        img = plt.imread(f"ShadowGP/extracted_results/{img_id}-output.png")[:,:,:3]
    elif method == "BlindShadowRemoval":
        img = plt.imread(f"BlindShadowRemoval/test/{img_id.split('-')[0]}_{img_id}-result.png")[:,:,:3]
        img = img[:236, 1280:1516, :]
        gt = plt.imread(f"gt/{img_id}.png")[:,:,:3]
        img = align_images(gt, img)
    elif method == "classical":
        img = plt.imread(f"classical/results/{img_id}.png")[:,:,:3]
    else:
        return ValueError("Method not implemented")
    if img.dtype == np.uint8:
        img = img / 255
    return img

def display_images(images, titles=None):
    plt.figure(figsize=(2.5 * len(images), 2.5))
    for i in range(len(images)):
        plt.subplot(1, len(images), i+1)
        if titles is not None:
            plt.title(titles[i])
        plt.imshow(images[i])
        plt.axis('off')
    plt.tight_layout()

def save_comparison(image_sets, titles=None):
    with PdfPages('comparison.pdf') as pdf:
        for image_set in image_sets:
            display_images(image_set, titles)
            pdf.savefig()
            plt.close()
    # compress pdf
    os.system("gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.4 -dNOPAUSE -dQUIET -dBATCH -sOutputFile=comparison_compressed.pdf comparison.pdf")
    os.system("rm comparison.pdf && mv comparison_compressed.pdf comparison.pdf")


In [17]:
image_ids = [path.split("/")[1].split(".")[0] for path in glob.glob("images/*.png")]
image_ids.sort()

image_sets = []
for image_index in range(len(image_ids)):
    image_id = image_ids[image_index]

    try:
        input_image = plt.imread(f"images/{image_id}.png")
        gt = plt.imread(f"gt/{image_id}.png")
        baseline = get_result(image_id, "baseline")
        bsr = get_result(image_id, "BlindShadowRemoval")
        shadowgp = get_result(image_id, "ShadowGP")
        classical = get_result(image_id, "classical")
        image_sets.append([input_image, baseline, bsr, classical, shadowgp, gt])
    except:
        pass
    
save_comparison(image_sets, ["Input", "Baseline", "BSR", "Classical (Ours)", "ShadowGP", "Ground Truth"])